## 添加Slice timing字段到json文件

In [2]:
import json, math
import numpy as np
import os
from bids import BIDSLayout

In [67]:
def slice_time(Num,TR):
    a =list(np.linspace(0, TR-TR/Num, Num))
    n = math.ceil(Num/2)
    b = a[0:n]
    c = a[n:]
    for i,item in enumerate(c):
        b.insert(i*2+1,item)
    return b

In [3]:
layout = BIDSLayout('/mnt/d/wangsiyu/Data/Visual_search_task/BIDS/Nifti')
json_list = layout.get(suffix='bold', extension='json', return_type='file')
for json_path in json_list:
    with open(json_path, 'rb') as f:
        json_file = json.load(f)
    json_file['SliceTiming'] = slice_time(31,2)
    with open(json_path, 'w') as file_obj:
        json.dump(json_file,file_obj, indent=4)

## 进行Fmriprep预处理

In [1]:
base_dir = '/mnt/d/wangsiyu/'
data_path = os.path.join(base_dir, 'Data/Visual_search_task/BIDS/Nifti')
output_path = os.path.join('/mnt/g/Result/Visual_search_task/fmriprep/derivatives')
work_path = os.path.join('/mnt/g/Result/Visual_search_task/fmriprep/work')
fs_path = os.path.join(base_dir, 'Result/Visual_search_task/license.txt')
subj_list = ['%02d'%i for i in range(18, 23)]
for sub in subj_list:
    !/home/kumadaken/.local/bin/fmriprep-docker {data_path} {output_path} --cifti-output --output-spaces MNI152NLin2009cAsym fsaverage5 --participant_label sub-{sub} --fs-license-file {fs_path} --work-dir {work_path}

RUNNING: docker run --rm -e DOCKER_VERSION_8395080871=20.10.11 -it -v /mnt/d/wangsiyu/Result/Visual_search_task/license.txt:/opt/freesurfer/license.txt:ro -v /mnt/d/wangsiyu/Data/Visual_search_task/BIDS/Nifti:/data:ro -v /mnt/g/Result/Visual_search_task/fmriprep/derivatives:/out -v /mnt/g/Result/Visual_search_task/fmriprep/work:/scratch nipreps/fmriprep:20.2.6 /data /out participant --cifti-output --participant_label sub-18 -w /scratch --output-spaces MNI152NLin2009cAsym fsaverage5
bids-validator@1.4.0

This dataset appears to be BIDS compatible.

        Summary:                  Available Tasks:        Available Modalities: 
        354 Files, 59.35GB        visualsearch            T1w                   
        22 - Subjects             rest                    bold                  
        5 - Sessions                                      events                


	If you have any questions, please post on https://neurostars.org/tags/bids.

211211-06:55:26,426 nipype.workflow WARNIN

In [10]:
base_dir = '/mnt/d/wangsiyu/'
data_path = os.path.join(base_dir, 'Data/ds000003')
output_path = os.path.join('/mnt/d/wangsiyu/Data/ds000003/derivatives')
work_path = os.path.join('/mnt/d/wangsiyu/Data')
fs_path = os.path.join(base_dir, 'Result/Visual_search_task/fmriprep/license.txt')
subj_list = ['%02d'%i for i in range(1, 2)]
for sub in subj_list:
    !/home/kumadaken/.local/bin/fmriprep-docker --fs-license-file {fs_path} --cifti-output --output-spaces MNI152NLin2009cAsym fsaverage5 --work-dir {work_path} {data_path} {output_path} --participant_label sub-{sub}

RUNNING: docker run --rm -e DOCKER_VERSION_8395080871=20.10.11 -it -v /mnt/d/wangsiyu/Result/Visual_search_task/fmriprep/license.txt:/opt/freesurfer/license.txt:ro -v /mnt/d/wangsiyu/Data/ds000003:/data:ro -v /mnt/d/wangsiyu/Data/ds000003/derivatives:/out -v /mnt/d/wangsiyu/Data:/scratch nipreps/fmriprep:20.2.6 /data /out participant --cifti-output --participant_label sub-01 -w /scratch --output-spaces MNI152NLin2009cAsym fsaverage5
bids-validator@1.4.0

	1: [WARN] You should define 'SliceTiming' for this file. If you don't provide this information slice time correction will not be possible. (code: 13 - SLICE_TIMING_NOT_DEFINED)
		./sub-01/func/sub-01_task-rhymejudgment_bold.nii.gz

	Please visit https://neurostars.org/search?q=SLICE_TIMING_NOT_DEFINED for existing conversations about this issue.


        Summary:                  Available Tasks:        Available Modalities: 
        57 Files, 391.09MB        rhyme judgment          T1w                   
        13 - Subjects       

## MRIQC质量控制代码

In [ ]:
docker run -it --rm \
	-v /mnt/d/wangsiyu/Data/Visual_search_task/BIDS/Nifti:/data:ro \
	-v /mnt/d/wangsiyu/Result/Visual_search_task/derivatives:/out \
	nipreps/mriqc:latest /data /out \
	participant --participant_label `ls /mnt/d/wangsiyu/Data/Visual_search_task/BIDS/Nifti|grep "sub-*"`